<a href="https://colab.research.google.com/github/Romqa41/GPN/blob/main/%D0%AD%D0%A2%D1%80%D0%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Блок импорта библиотек

In [ ]:
pip install natasha

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 111.2 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=2b2de9e97808695f00ac7eeb359ea846b05fbeeca64680e3f8cfa48ab9b62e36
  Stored in directory: /root/.cache/pip/wheels/1a/bf/a1/4cee4f7678c68c5875ca89eaccf460593539805c3906722228
Successfully built docopt


In [ ]:
import pandas as pd
import numpy as np
import re
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    Doc
    )

# Блок загрузки данных

In [ ]:
# журнал прихода (ZINMM521)
arr_journal = pd.read_excel('Журнал прихода ZINMM521 - ЭТрН.xlsx')

# выгрузка ЭТрН (ZDDK_WAYBILL)
invoices = pd.read_excel('Выгрузка по ЭТрН.xlsx', header=None)

# M11
m11 = pd.read_excel('ЭД_М11.XLSX', dtype={'Завод': str})

# дивизионы
divisions = pd.read_excel('Дивизионы и ОП.xlsx')

# правила
manual = pd.read_excel('Справочник.xlsx')

# титулы 1-4
title_1 = pd.read_excel('Титул 1.xlsx')
title_2 = pd.read_excel('Титул 2.xlsx')
title_3 = pd.read_excel('Титул 3.xlsx')
title_4 = pd.read_excel('Титул 4.xlsx')

# архив накладных из Контура
archive = pd.read_excel('Подписано.xlsx')

/usr/local/lib/python3.12/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.12/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.12/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.12/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.12/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


# Блок функций

In [ ]:
# инициализация компонентов, которые нужны для поиска ФИО в перевозчиках
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

Поиск ФИО

In [ ]:
def fio_check(text):
  # все слова с большой буквы
  text = text.title()

  # создание объекта документа для анализа (выполняет роль контейнера для хранения токенов и рез-ов анализа)
  doc = Doc(text)

  # выполнение анализа - разбиение на токены, морфологический и синтаксический анализ, распознавание именованных сущностей
  doc.segment(segmenter)
  doc.tag_morph(morph_tagger)
  doc.parse_syntax(syntax_parser)
  doc.tag_ner(ner_tagger)

  # проверка на основные формы ЮЛ
  if re.search(r'(ООО|ЗАО|ИП|ГУП|ГПН)', text, flags=re.IGNORECASE):
    return 0

  # поиск признаков ФИО
  for span in doc.spans:
    if span.type == 'PER':
      return 1

  # если признаки не найдены, то это не ФИО
  return 0

Нормализация перевозчиков

In [ ]:
def normalize_contractor(text):
  # проверка типа данных (должен быть строковый)
  if isinstance(text, str):

    # удаление номера ИНН вместе с предшествующим словом "ИНН"
    cleaned_text = re.sub(r'\bИНН\s*\d+\b', '', text)
    # замещение всех знаков препинания и спецсимволов кроме пробела
    cleaned_text = re.sub(r'[^\w\s]', '', cleaned_text)
    # нормализация пробелов (остаются только одиночные)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()

    # основные формы ЮЛ
    legal_forms = ['ООО', 'АО', 'ЗАО', 'ОАО', 'ИП', 'ГУП', 'МУП']

    # добавление пробелов после формы ЮЛ
    pattern = '(' + '|'.join(re.escape(form) for form in legal_forms) + ')(\w)'
    cleaned_text = re.sub(pattern, r'\1 \2', cleaned_text)

    # приведение текста к верхнему регистру
    return cleaned_text.upper()
  else:
    return ''

<>:16: SyntaxWarning: invalid escape sequence '\w'
<>:16: SyntaxWarning: invalid escape sequence '\w'
/tmp/ipython-input-1472920421.py:16: SyntaxWarning: invalid escape sequence '\w'
  pattern = '(' + '|'.join(re.escape(form) for form in legal_forms) + ')(\w)'


Общая обработка данных

In [ ]:
def data_processing(df, rules):

  # работаем в копии исходного датафрейма
  cleared_data = df.copy()

  cleared_data = cleared_data[cleared_data['Организация перевозчик'].notna()]

  # проходим циклом по справочнику
  for idx, row in rules.iterrows():

    # получаем название столбца
    col = row['Поле']

    # фиксируем то, что требует обработки / удаления
    content = row['Формат']

    # фиксируем правило
    rule = row['Правило']

    # удаляем строку по нахождению строго отдельного слова
    if rule == 'Удалить отдельно':
      pattern = rf'\b{re.escape(content)}\b'

    # удаляем строку по нахождению НЕ строго отдельного слова
    elif rule == 'Удалить':
      pattern = rf'{content}'

    # удаляем строку по заданному окончанию
    elif rule == 'Удалить окончание':
      pattern = rf'\b\d{{3}}{re.escape(content)}\b'

    # удаляем строку по нахождению двух отдельных слов
    elif rule == 'Удалить оба':
      words = re.split(r'[., ]+', content)
      pattern = '|'.join(fr'\b{re.escape(word)}\b' for word in words)

    # применяем полученную маску для фильтрации датафрейма
    cleared_data = cleared_data[~cleared_data[col].astype(str).str.contains(pattern, case=False, regex=True)]

  return cleared_data

# Блок рабочего кода

In [ ]:
# считаем кол-во повторов документа
arr_journal['Повтор документа'] = arr_journal.groupby('Документ материала').cumcount()+1

# оставляем только строки с первым нахождением номера документа, дубли удаляем
arr_journal = arr_journal[arr_journal['Повтор документа'] == 1]

# удаляем все строки с пропущенным документом
m11 = m11[m11['Документ-основание'].notna()]

In [ ]:
# для всех заводов фиксируем формат - четыре цифры, по необходимости дополнеяем ведущими нулями
arr_journal['Принимающий завод'] = arr_journal['Принимающий завод'].astype(str).str.zfill(4)
arr_journal['Отпускающий завод'] = arr_journal['Отпускающий завод'].astype(str).str.zfill(4)

divisions['Завод'] = divisions['Завод'].astype(str).str.zfill(4)

m11['Завод'] = m11['Завод'].astype(str).str.zfill(4)

In [ ]:
# используем функцию обработки данных - удаляем строки согласно правилам справочника
journal_ed = data_processing(arr_journal, manual)
journal_ed

,Отпускающий завод,Принимающий завод,Отпускающий склад,Принимающий склад,Отпускающий склад.1,Принимающий склад.1,Дата документа,Имя пользователя,Документ материала,№ заказа на перемещение,...,Государственный номер автомашины,Фамилия водителя,Вид движения,Тоннаж,Класс оценки,Бух.Статус утв.,Текст заголовка документа,Транспортная накладная,Вид докум. закупки,Повтор документа
160,1241,1241,0127,1120,пос. Новый Порт,НП ГСМ/Химпр/Газ,2025-09-29,SOTNIKOVSI,4914914205,4501858689,...,С763ХО55,ЯУК,351,21.914,4113,X,NaN,NaN,ZUB,1
162,1241,1241,0127,1120,пос. Новый Порт,НП ГСМ/Химпр/Газ,2025-09-29,SOTNIKOVSI,4914914263,4501858689,...,М748МР18,ДЖАММАВОВ,351,24.802,4113,X,NaN,NaN,ZUB,1
163,1241,1241,0127,1120,пос. Новый Порт,НП ГСМ/Химпр/Газ,2025-09-29,SOTNIKOVSI,4914914266,4501858689,...,У362НМ55,АФЕЛЬД,351,25.226,4113,X,NaN,NaN,ZUB,1
270,1021,1021,0170,0109,Транз.ст.Демьянк,ЦСЛ №1 (Причал),2025-09-29,GUSEVNV,4914914397,4501857902,...,У519КВ774,ВИКТОРКИН,351,6.342,1700,X,NaN,NaN,ZUB,1
325,4071,4071,ТЮ68,ТЮ67,Мегатранс РЦ,УТК РЦ,2025-09-29,BARANOVEL,4914914433,4501859752,...,Т898ХС174,ГАЛИАСКАРОВ,351,20.400,4115,NaN,"ПА 5397,5398,5399,5400",NaN,ZUB,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33722,4071,4071,ТЮ68,ТЮ90,Мегатранс РЦ,МегатрансРЦР,2025-10-12,BARANOVEL,4914961316,4501866404,...,В667ХУ18,БЕРДОВ,351,8.218,4115,NaN,Исх/п.№НС-03.3.3/3484,NaN,ZUB,1
33895,1241,1241,0127,0104,пос. Новый Порт,НП №4 Новый Порт,2025-10-12,BYRDINMV,4914961419,4501867065,...,Т065ОМ55,КИРИЧЕНКО,351,4.000,20,NaN,"№38(40), пл.05945062",NaN,ZUB,1
34016,4071,4071,ТЮ68,ТЮ90,Мегатранс РЦ,МегатрансРЦР,2025-10-12,BARANOVEL,4914961454,4501866055,...,В492ВА774,МОИСЕЕНКО,351,20.828,4115,NaN,Исх/п.№НС-03.3.3/3263,NaN,ZUB,1
34020,4071,4071,ТЮ68,ТЮ67,Мегатранс РЦ,УТК РЦ,2025-10-12,BARANOVEL,4914961457,4501863466,...,Т898ХС174,ГАЛИАСКАРОВ,351,20.400,4115,NaN,"ПА 5341,5342,5343,5344",NaN,ZUB,1


In [ ]:
# проверка перевозчиков на ФИО (1 - да, 0 - нет)
journal_ed['ФИО'] = journal_ed['Организация перевозчик'].apply(fio_check)

In [ ]:
# удаляем строки с ФИО в поле перевозчиков
journal_ed = journal_ed[journal_ed['ФИО'] == 0]

In [ ]:
# присоединяем к очищенной ZINMM521 выгрузку с ЭТрН ZDDK_WAYBILL
journal_ed = journal_ed.merge(invoices[6], how='left', left_on='Документ материала', right_on=6)

# переименовываем столбец
journal_ed = journal_ed.rename(columns={6:'Найд. док-ты'})

In [ ]:
# присоединяем М11
journal_ed = journal_ed.merge(m11['Документ-основание'], how='left', left_on='Документ материала', right_on='Документ-основание')

In [ ]:
# факт наличия ЭТрН (1 - да, 0 - нет)
journal_ed['Наличие ЭТрН'] = np.where(journal_ed['Найд. док-ты'].isna(), 0, 1)

# факт наличия М11 (1 - да, 0 - нет)
journal_ed['Наличие М11'] = np.where(journal_ed['Документ-основание'].isna(), 0, 1)

In [ ]:
# присоединяем дивизионы и ОП к созданной таблице
final_docs = journal_ed.merge(divisions, how='left',
                              left_on=['Отпускающий завод', 'Отпускающий склад', 'Отпускающий склад.1'],
                              right_on=['Завод', 'Склад (номер)', 'Склад (название)']
                              )

# отсекаем лишние столбцы
final_docs = final_docs[[
    'Дивизион',
    'ОП',
    'Отпускающий склад.1',
    'Дата документа',
    'Имя пользователя',
    'Документ материала',
    '№ заказа на перемещение',
    'Краткий текст материала',
    'Организация перевозчик',
    'Государственный номер автомашины',
    'Фамилия водителя',
    # 'Найд. док-ты',
    # 'Документ-основание',
    'Наличие ЭТрН',
    'Наличие М11'
    ]]

# переименовываем строки
final_docs = final_docs.rename(columns={
    'Отпускающий склад.1':'Отпускающий склад',
    'Дата документа':'Дата',
    'Имя пользователя':'Пользователь',
    'Документ материала':'№ документа',
    '№ заказа на перемещение':'Заявка',
    'Краткий текст материала':'Материал',
    'Организация перевозчик':'Перевозчик',
    'Государственный номер автомашины':'Номер АТ'
    })

In [ ]:
# нормализуем наименования перевозчиков
final_docs['Перевозчик'] = final_docs['Перевозчик'].apply(normalize_contractor)

In [ ]:
# добавляем статус поездки (если завершена - дивизион, если нет - "В пути")
final_docs['Завершение поездки'] = np.where(final_docs['Наличие М11'] == 1, final_docs['Дивизион'], 'В пути')

In [ ]:
# группируем данные по дивизионам, ОП и перевозчикам
final_groups = (
    final_docs
    .groupby(['Дивизион', 'ОП', 'Перевозчик']).agg({
        '№ документа':'count',
        'Наличие ЭТрН':'sum',
        'Наличие М11':'sum'
    })
    .rename(columns={
        '№ документа':'Кол-во ТН',
        'Наличие ЭТрН':'Кол-во ЭТрН',
        'Наличие М11':'Подписанные М11'
    })
    .assign(**{'Процент кол-ва ЭТрН к общему кол-ву перемещений':lambda x: round(x['Кол-во ЭТрН'] / x['Кол-во ТН'] * 100, 0)})
    .reset_index()
)

# приводим данные столбца к целочисленному типу
final_groups['Процент кол-ва ЭТрН к общему кол-ву перемещений'] = (
    final_groups['Процент кол-ва ЭТрН к общему кол-ву перемещений'].astype(int)
)

# меняем порядок столбцов
final_groups = final_groups[[
    'Дивизион',
    'ОП',
    'Перевозчик',
    'Кол-во ТН',
    'Кол-во ЭТрН',
    'Процент кол-ва ЭТрН к общему кол-ву перемещений',
    'Подписанные М11'
    ]]

final_groups

,Дивизион,ОП,Перевозчик,Кол-во ТН,Кол-во ЭТрН,Процент кол-ва ЭТрН к общему кол-ву перемещений,Подписанные М11
0,?,?,АО АВРОРА ЛОГИСТИКА,8,1,12,0
1,?,?,ГПНДОРОЖНОЕ СИРОИТЕЛЬСТВО,1,0,0,0
2,?,?,ООО 2ЖЕЛДОРЭКСЕДИЦИЯ,1,0,0,1
3,?,?,ООО АВТОТЕХСЕРВИС,1,1,100,0
4,?,?,ООО АРМАДА,2,0,0,0
5,?,?,ООО ВОЛГА ТРАНСОЙЛ,1,0,0,1
6,?,?,ООО ДЛТРАНС,1,0,0,0
7,?,?,ООО ЖЕЛДОРЭКСПЕДИЦИЯ,1,0,0,0
8,?,?,ООО НИТРОЛОГИСТИК,1,0,0,0
9,?,?,ООО НПО РИЗУР,8,0,0,3


In [ ]:
# информация по дивизионам
divs_table = (
    final_docs
    .groupby('Дивизион')
    .agg({'№ документа': 'count', 'Наличие ЭТрН': 'sum', 'Наличие М11':'sum'})
    .reset_index()
    .rename(columns={
        '№ документа':'Общее количество межскладских перемещений',
        'Наличие ЭТрН':'Кол-во ЭТрН',
        'Наличие М11':'Кол-во М11'
        })
)

divs_table

,Дивизион,Общее количество межскладских перемещений,Кол-во ЭТрН,Кол-во М11
0,?,39,5,14
1,Восток,4,0,4
2,Восточная Сибирь,37,13,25
3,Север,95,91,90
4,Центр,54,39,47


In [ ]:
# дополненный архив
archive_mod = archive.merge(
    final_docs[['№ документа', 'Дивизион', 'Завершение поездки']].rename({'Завершение поездки':'Подписано дивизионом'}),
    left_on='Номер ТРН',
    right_on='№ документа',
    how='left'
    ).drop('№ документа', axis=1)

# проверяем наличие дубликата в роли "Грузоотправитель", графа "Перевозчик" пустая
archive_mod = archive_mod.merge(
    title_1[['Номер ТРН']].rename(columns={'Номер ТРН':'Дубликат Т1'}), left_on='Номер ТРН', right_on='Дубликат Т1', how='left'
    )

# проверяем наличие дубликата в роли "Грузоотправитель", графа "Перевозчик" заполнена
archive_mod = archive_mod.merge(
    title_2[['Номер ТРН']].rename(columns={'Номер ТРН':'Дубликат Т2'}), left_on='Номер ТРН', right_on='Дубликат Т2', how='left'
    )

# проверяем наличие дубликата в роли "Грузополучатель", разделе "Отгруженные"
archive_mod = archive_mod.merge(
    title_3[['Номер ТРН']].rename(columns={'Номер ТРН':'Дубликат Т3'}), left_on='Номер ТРН', right_on='Дубликат Т3', how='left'
    )

# проверяем наличие дубликата в роли "Грузополучатель", разделе "На подпись"
archive_mod = archive_mod.merge(
    title_4[['Номер ТРН']].rename(columns={'Номер ТРН':'Дубликат Т4'}), left_on='Номер ТРН', right_on='Дубликат Т4', how='left'
    )

In [ ]:
# дополненный титул 1
title_1_mod = title_1.merge(
    final_docs[['№ документа', 'Дивизион', 'Завершение поездки']].rename({'Завершение поездки':'Принято дивизионом'}),
    left_on='Номер ТРН',
    right_on='№ документа',
    how='left'
    ).drop('№ документа', axis=1)

# проверяем наличие дубликата в роли "Грузоотправитель", графа "Перевозчик" заполнена
title_1_mod = title_1_mod.merge(
    title_2[['Номер ТРН']].rename(columns={'Номер ТРН':'Дубликат Т2'}), left_on='Номер ТРН', right_on='Дубликат Т2', how='left'
    )

# проверяем наличие дубликата в роли "Грузополучатель", разделе "Отгруженные"
title_1_mod = title_1_mod.merge(
    title_3[['Номер ТРН']].rename(columns={'Номер ТРН':'Дубликат Т3'}), left_on='Номер ТРН', right_on='Дубликат Т3', how='left'
    )

# проверяем наличие дубликата в роли "Грузополучатель", разделе "На подпись"
title_1_mod = title_1_mod.merge(
    title_4[['Номер ТРН']].rename(columns={'Номер ТРН':'Дубликат Т4'}), left_on='Номер ТРН', right_on='Дубликат Т4', how='left'
    )

# проверяем наличие дубликата в роли "Грузополучатель", разделе "На подпись"
title_1_mod = title_1_mod.merge(
    archive[['Номер ТРН']].rename(columns={'Номер ТРН':'Дубликат Подписано'}), left_on='Номер ТРН', right_on='Дубликат Подписано', how='left'
    )

In [ ]:
# дополненный титул 2
title_2_mod = title_2.merge(
    final_docs[['№ документа', 'Дивизион', 'Завершение поездки']].rename({'Завершение поездки':'Принято дивизионом'}),
    left_on='Номер ТРН',
    right_on='№ документа',
    how='left'
    ).drop('№ документа', axis=1)

# проверяем наличие дубликата в роли "Грузополучатель", разделе "Отгруженные"
title_2_mod = title_2_mod.merge(
    title_3[['Номер ТРН']].rename(columns={'Номер ТРН':'Дубликат Т3'}), left_on='Номер ТРН', right_on='Дубликат Т3', how='left'
    )

# проверяем наличие дубликата в роли "Грузополучатель", разделе "На подпись"
title_2_mod = title_2_mod.merge(
    title_4[['Номер ТРН']].rename(columns={'Номер ТРН':'Дубликат Т4'}), left_on='Номер ТРН', right_on='Дубликат Т4', how='left'
    )

# проверяем наличие дубликата в роли "Грузополучатель", разделе "На подпись"
title_2_mod = title_2_mod.merge(
    archive[['Номер ТРН']].rename(columns={'Номер ТРН':'Дубликат Подписано'}), left_on='Номер ТРН', right_on='Дубликат Подписано', how='left'
    )

In [ ]:
# дополненный титул 3
title_3_mod = title_3.merge(
    final_docs[['№ документа', 'Дивизион', 'Завершение поездки']].rename({'Завершение поездки':'Принято дивизионом'}),
    left_on='Номер ТРН',
    right_on='№ документа',
    how='left'
    ).drop('№ документа', axis=1)

# проверяем наличие дубликата в роли "Грузополучатель", разделе "На подпись"
title_3_mod = title_3_mod.merge(
    title_4[['Номер ТРН']].rename(columns={'Номер ТРН':'Дубликат Т4'}), left_on='Номер ТРН', right_on='Дубликат Т4', how='left'
    )

# проверяем наличие дубликата в роли "Грузополучатель", разделе "На подпись"
title_3_mod = title_3_mod.merge(
    archive[['Номер ТРН']].rename(columns={'Номер ТРН':'Дубликат Подписано'}), left_on='Номер ТРН', right_on='Дубликат Подписано', how='left'
    )

In [ ]:
# дополненный титул 4
title_4_mod = title_4.merge(
    final_docs[['№ документа', 'Дивизион', 'Завершение поездки']].rename({'Завершение поездки':'Принято дивизионом'}),
    left_on='Номер ТРН',
    right_on='№ документа',
    how='left'
    ).drop('№ документа', axis=1)

# проверяем наличие дубликата в роли "Грузополучатель", разделе "На подпись"
title_4_mod = title_4_mod.merge(
    archive[['Номер ТРН']].rename(columns={'Номер ТРН':'Дубликат Подписано'}), left_on='Номер ТРН', right_on='Дубликат Подписано', how='left'
    )

In [ ]:
# формируем таблицу с дивизионами
archive_gr = pd.DataFrame({
    'Подписано': ['Восточная Сибирь', 'Север', 'Центр', 'Восток']
})

# подсчет количества перевозок и завершенных перевозок в дополненном архиве
freq_all = archive_mod['Дивизион'].value_counts().to_frame(name='ЭТрН в Контуре').reset_index()
freq_m11 = archive_mod['Завершение поездки'].value_counts().to_frame(name='Из них завершено поездок').reset_index()

# присоединяем рассчитанное кол-во перевозок в дополненном архиве
archive_gr = (
    archive_gr
    .merge(freq_all, left_on='Подписано', right_on='Дивизион', how='left')
    .drop('Дивизион', axis=1)
)

# присоединяем рассчитанное кол-во завершенных перевозок в дополненном архиве
archive_gr = (
    archive_gr
    .merge(freq_m11, left_on='Подписано', right_on='Завершение поездки', how='left')
    .drop('Завершение поездки', axis=1)
)

# добавляем строку ИТОГО
total_row = {
    'Подписано': 'Итого',
    'ЭТрН в Контуре': archive_gr['ЭТрН в Контуре'].sum(),
    'Из них завершено поездок': archive_gr['Из них завершено поездок'].sum()
    }

# подсчет длины сгруппированного дополненного архива
archive_gr.loc[len(archive_gr)] = total_row

# заполняем пропуски нулями
archive_gr = archive_gr.fillna(0)

# численные значения приводим к целочисленныму типу данных
archive_gr = archive_gr.astype({'ЭТрН в Контуре': int, 'Из них завершено поездок': int})

archive_gr

,Подписано,ЭТрН в Контуре,Из них завершено поездок
0,Восточная Сибирь,0,0
1,Север,85,84
2,Центр,2,2
3,Восток,0,0
4,Итого,87,86


In [ ]:
# формируем таблицу с дивизионами
title_1_gr = pd.DataFrame({
    'Титул 1': ['Восточная Сибирь', 'Север', 'Центр', 'Восток']
})

# подсчет количества перевозок и завершенных перевозок в дополненном титуле 1
freq_all = title_1_mod['Дивизион'].value_counts().to_frame(name='ЭТрН в Контуре').reset_index()
freq_m11 = title_1_mod['Завершение поездки'].value_counts().to_frame(name='Из них завершено поездок').reset_index()

# присоединяем рассчитанное кол-во перевозок в дополненном титуле 1
title_1_gr = (
    title_1_gr
    .merge(freq_all, left_on='Титул 1', right_on='Дивизион', how='left')
    .drop('Дивизион', axis=1)
)

# присоединяем рассчитанное кол-во завершенных перевозок в дополненном титуле 1
title_1_gr = (
    title_1_gr
    .merge(freq_m11, left_on='Титул 1', right_on='Завершение поездки', how='left')
    .drop('Завершение поездки', axis=1)
)

# добавляем строку ИТОГО
total_row = {
    'Титул 1': 'Итого',
    'ЭТрН в Контуре': title_1_gr['ЭТрН в Контуре'].sum(),
    'Из них завершено поездок': title_1_gr['Из них завершено поездок'].sum()
    }

# подсчет длины сгруппированного дополненного титула 1
title_1_gr.loc[len(title_1_gr)] = total_row

# заполняем пропуски нулями
title_1_gr = title_1_gr.fillna(0)

# численные значения приводим к целочисленныму типу данных
title_1_gr = title_1_gr.astype({'ЭТрН в Контуре': int, 'Из них завершено поездок': int})

title_1_gr

,Титул 1,ЭТрН в Контуре,Из них завершено поездок
0,Восточная Сибирь,1,1
1,Север,0,0
2,Центр,12,12
3,Восток,0,0
4,Итого,13,13


In [ ]:
# формируем таблицу с дивизионами
title_2_gr = pd.DataFrame({
    'Титул 2': ['Восточная Сибирь', 'Север', 'Центр', 'Восток']
})

# подсчет количества перевозок и завершенных перевозок в дополненном титуле 2
freq_all = title_2_mod['Дивизион'].value_counts().to_frame(name='ЭТрН в Контуре').reset_index()
freq_m11 = title_2_mod['Завершение поездки'].value_counts().to_frame(name='Из них завершено поездок').reset_index()

# присоединяем рассчитанное кол-во перевозок в дополненном титуле 2
title_2_gr = (
    title_2_gr
    .merge(freq_all, left_on='Титул 2', right_on='Дивизион', how='left')
    .drop('Дивизион', axis=1)
)

# присоединяем рассчитанное кол-во завершенных перевозок в дополненном титуле 2
title_2_gr = (
    title_2_gr
    .merge(freq_m11, left_on='Титул 2', right_on='Завершение поездки', how='left')
    .drop('Завершение поездки', axis=1)
)

# добавляем строку ИТОГО
total_row = {
    'Титул 2': 'Итого',
    'ЭТрН в Контуре': title_2_gr['ЭТрН в Контуре'].sum(),
    'Из них завершено поездок': title_2_gr['Из них завершено поездок'].sum()
    }

# подсчет длины сгруппированного дополненного титула 2
title_2_gr.loc[len(title_2_gr)] = total_row

# заполняем пропуски нулями
title_2_gr = title_2_gr.fillna(0)

# численные значения приводим к целочисленныму типу данных
title_2_gr = title_2_gr.astype({'ЭТрН в Контуре': int, 'Из них завершено поездок': int})

title_2_gr

,Титул 2,ЭТрН в Контуре,Из них завершено поездок
0,Восточная Сибирь,15,11
1,Север,8,4
2,Центр,8,8
3,Восток,0,0
4,Итого,31,23


In [ ]:
# формируем таблицу с дивизионами
title_3_gr = pd.DataFrame({
    'Титул 3': ['Восточная Сибирь', 'Север', 'Центр', 'Восток']
})

# подсчет количества перевозок и завершенных перевозок в дополненном титуле 3
freq_all = title_3_mod['Дивизион'].value_counts().to_frame(name='ЭТрН в Контуре').reset_index()
freq_m11 = title_3_mod['Завершение поездки'].value_counts().to_frame(name='Из них завершено поездок').reset_index()

# присоединяем рассчитанное кол-во перевозок в дополненном титуле 3
title_3_gr = (
    title_3_gr
    .merge(freq_all, left_on='Титул 3', right_on='Дивизион', how='left')
    .drop('Дивизион', axis=1)
)

# присоединяем рассчитанное кол-во завершенных перевозок в дополненном титуле 3
title_3_gr = (
    title_3_gr
    .merge(freq_m11, left_on='Титул 3', right_on='Завершение поездки', how='left')
    .drop('Завершение поездки', axis=1)
)

# добавляем строку ИТОГО
total_row = {
    'Титул 3': 'Итого',
    'ЭТрН в Контуре': title_3_gr['ЭТрН в Контуре'].sum(),
    'Из них завершено поездок': title_3_gr['Из них завершено поездок'].sum()
    }

# подсчет длины сгруппированного дополненного титула 3
title_3_gr.loc[len(title_3_gr)] = total_row

# заполняем пропуски нулями
title_3_gr = title_3_gr.fillna(0)

# численные значения приводим к целочисленныму типу данных
title_3_gr = title_3_gr.astype({'ЭТрН в Контуре': int, 'Из них завершено поездок': int})

title_3_gr

,Титул 3,ЭТрН в Контуре,Из них завершено поездок
0,Восточная Сибирь,0,0
1,Север,2,2
2,Центр,18,15
3,Восток,0,0
4,Итого,20,17


In [ ]:
# формируем таблицу с дивизионами
title_4_gr = pd.DataFrame({
    'Титул 4': ['Восточная Сибирь', 'Север', 'Центр', 'Восток']
})

# подсчет количества перевозок и завершенных перевозок в дополненном титуле 4
freq_all = title_4_mod['Дивизион'].value_counts().to_frame(name='ЭТрН в Контуре').reset_index()
freq_m11 = title_4_mod['Завершение поездки'].value_counts().to_frame(name='Из них завершено поездок').reset_index()

# присоединяем рассчитанное кол-во перевозок в дополненном титуле 4
title_4_gr = (
    title_4_gr
    .merge(freq_all, left_on='Титул 4', right_on='Дивизион', how='left')
    .drop('Дивизион', axis=1)
)

# присоединяем рассчитанное кол-во завершенных перевозок в дополненном титуле 4
title_4_gr = (
    title_4_gr
    .merge(freq_m11, left_on='Титул 4', right_on='Завершение поездки', how='left')
    .drop('Завершение поездки', axis=1)
)

# добавляем строку ИТОГО
total_row = {
    'Титул 4': 'Итого',
    'ЭТрН в Контуре': title_4_gr['ЭТрН в Контуре'].sum(),
    'Из них завершено поездок': title_4_gr['Из них завершено поездок'].sum()
    }

# подсчет длины сгруппированного дополненного титула 4
title_4_gr.loc[len(title_4_gr)] = total_row

# заполняем пропуски нулями
title_4_gr = title_4_gr.fillna(0)

# численные значения приводим к целочисленныму типу данных
title_4_gr = title_4_gr.astype({'ЭТрН в Контуре': int, 'Из них завершено поездок': int})

title_4_gr

,Титул 4,ЭТрН в Контуре,Из них завершено поездок
0,Восточная Сибирь,0,0
1,Север,0,0
2,Центр,0,0
3,Восток,0,0
4,Итого,0,0


In [ ]:
# итоговый свод по дивизонам (аналог вкладки "счет (2)" в Excel)
final_table = (
    pd.merge(divs_table, archive_gr, left_on='Дивизион', right_on='Подписано', how='left')
    .drop(['Подписано', 'ЭТрН в Контуре'], axis=1)
    .rename(columns={'Из них завершено поездок':'Кол-во подписанных ЭТрН по завершенным поездкам'})
    .assign(**{'Процент подписанных ЭТрН по завершенным поездкам':lambda x: np.where(x['Кол-во М11'] != 0, round(x['Кол-во подписанных ЭТрН по завершенным поездкам'] / x['Кол-во М11'] * 100, 0), 0)})
)

# сведение титулов 1-4 в одну таблицу
titles_1234_gr = (
    title_1_gr
    .merge(title_2_gr, left_on='Титул 1', right_on='Титул 2', how='left')
    .merge(title_3_gr, left_on='Титул 1', right_on='Титул 3', how='left')
    .drop(['ЭТрН в Контуре_x', 'ЭТрН в Контуре_y', 'ЭТрН в Контуре', 'Титул 2', 'Титул 3'], axis=1)
    .rename(columns={'Из них завершено поездок_x':'з1', 'Из них завершено поездок_y':'з2','Из них завершено поездок':'з3'})
    .merge(title_4_gr, left_on='Титул 1', right_on='Титул 4', how='left')
    .drop(['ЭТрН в Контуре', 'Титул 4'], axis=1)
    .rename(columns={'Из них завершено поездок':'з4'})
    .assign(з_1234=lambda x: x['з1'] + x['з2'] + x['з3'] + x['з4'])
    .drop(['з1', 'з2', 'з3', 'з4'], axis=1)
    .rename(columns={'з_1234':'Кол-во не подписанных ЭТрН по завершенным поездкам', 'Титул 1':'Дивизион'})
)

# подтягиваем сведенную информацию по титулам к итоговой таблице
final_table = (
    final_table.merge(titles_1234_gr, on='Дивизион', how='left')
    .assign(**{'Процент не подписанных ЭТрН по завершенным поездкам':lambda x: np.where(x['Кол-во М11'] != 0, round(x['Кол-во не подписанных ЭТрН по завершенным поездкам'] / x['Кол-во М11'] * 100, 0), 0)})
)

# формируем итоговую строку
total_row = {
    'Дивизион': 'Итого',
    'Общее количество межскладских перемещений': final_table['Общее количество межскладских перемещений'].sum(),
    'Кол-во ЭТрН': final_table['Кол-во ЭТрН'].sum(),
    'Кол-во подписанных ЭТрН по завершенным поездкам': final_table['Кол-во подписанных ЭТрН по завершенным поездкам'].sum(),
    'Процент подписанных ЭТрН по завершенным поездкам': np.where(final_table['Кол-во М11'].sum() != 0, round(final_table['Кол-во подписанных ЭТрН по завершенным поездкам'].sum() / final_table['Кол-во М11'].sum() * 100, 0), 0),
    'Кол-во не подписанных ЭТрН по завершенным поездкам': final_table['Кол-во не подписанных ЭТрН по завершенным поездкам'].sum(),
    'Процент не подписанных ЭТрН по завершенным поездкам': np.where(final_table['Кол-во М11'].sum() != 0, round(final_table['Кол-во не подписанных ЭТрН по завершенным поездкам'].sum() / final_table['Кол-во М11'].sum() * 100, 0), 0),
    }

# добавляем итоговую строку
final_table.loc[len(final_table)] = total_row

# заполняем пропуски нулями
final_table = final_table.fillna(0)

# приводим количественные данные к целочисленному типу
final_table = final_table.astype({
    'Кол-во подписанных ЭТрН по завершенным поездкам': int,
    'Кол-во не подписанных ЭТрН по завершенным поездкам': int,
    'Процент подписанных ЭТрН по завершенным поездкам': int,
    'Процент не подписанных ЭТрН по завершенным поездкам': int
    }).drop('Кол-во М11', axis=1)

# добавляем столбец отношения кол-ва ЭТрН к общему кол-ву перемещений
perc = round(final_table['Кол-во ЭТрН'] / final_table['Общее количество межскладских перемещений'] * 100, 0)
pos = final_table.columns.get_loc('Кол-во ЭТрН') + 1
final_table.insert(pos, 'Процент кол-ва ЭТрН к общему кол-ву перемещений', perc)

# приводим количественные данные к целочисленному типу
final_table = final_table.astype({'Процент кол-ва ЭТрН к общему кол-ву перемещений': int})

final_table

,Дивизион,Общее количество межскладских перемещений,Кол-во ЭТрН,Процент кол-ва ЭТрН к общему кол-ву перемещений,Кол-во подписанных ЭТрН по завершенным поездкам,Процент подписанных ЭТрН по завершенным поездкам,Кол-во не подписанных ЭТрН по завершенным поездкам,Процент не подписанных ЭТрН по завершенным поездкам
0,?,39,5,13,0,0,0,0
1,Восток,4,0,0,0,0,0,0
2,Восточная Сибирь,37,13,35,0,0,12,48
3,Север,95,91,96,84,93,6,7
4,Центр,54,39,72,2,4,35,74
5,Итого,229,148,65,86,48,53,29


In [ ]:
# формируем итоговую строку для таблицы дивизионов
total_row = {
    'Дивизион': 'Итого',
    'Общее количество межскладских перемещений': divs_table['Общее количество межскладских перемещений'].sum(),
    'Кол-во ЭТрН': divs_table['Кол-во ЭТрН'].sum(),
    'Кол-во М11': divs_table['Кол-во М11'].sum()
    }

# добавляем итоговую строку в таблицу дивизионов
divs_table.loc[len(divs_table)] = total_row

# заполняем пропуски нулями
divs_table = divs_table.fillna(0)

# итоговая таблица (аналог "счет (2)" в Excel)
divs_table_fin = divs_table.merge(
    final_table.drop(['Общее количество межскладских перемещений', 'Кол-во ЭТрН'], axis=1),
    on='Дивизион',
    how='left'
    ).drop('Кол-во М11', axis=1)

divs_table_fin

,Дивизион,Общее количество межскладских перемещений,Кол-во ЭТрН,Процент кол-ва ЭТрН к общему кол-ву перемещений,Кол-во подписанных ЭТрН по завершенным поездкам,Процент подписанных ЭТрН по завершенным поездкам,Кол-во не подписанных ЭТрН по завершенным поездкам,Процент не подписанных ЭТрН по завершенным поездкам
0,?,39,5,13,0,0,0,0
1,Восток,4,0,0,0,0,0,0
2,Восточная Сибирь,37,13,35,0,0,12,48
3,Север,95,91,96,84,93,6,7
4,Центр,54,39,72,2,4,35,74
5,Итого,229,148,65,86,48,53,29


In [ ]:
final_groups_briefly = divs_table_fin.copy()

final_groups_briefly['ЭТрН подписаны по завершенным поездкам'] = (
    final_groups_briefly['Процент подписанных ЭТрН по завершенным поездкам'].astype(str)
    + '% ('
    + final_groups_briefly['Кол-во подписанных ЭТрН по завершенным поездкам'].astype(str)
    + ' шт)'
)

final_groups_briefly['ЭТрН не подписаны по завершенным поездкам'] = (
    final_groups_briefly['Процент не подписанных ЭТрН по завершенным поездкам'].astype(str)
    + '% ('
    + final_groups_briefly['Кол-во не подписанных ЭТрН по завершенным поездкам'].astype(str)
    + ' шт)'
)

final_groups_briefly = final_groups_briefly.drop([
    'Кол-во подписанных ЭТрН по завершенным поездкам',
    'Процент подписанных ЭТрН по завершенным поездкам',
    'Кол-во не подписанных ЭТрН по завершенным поездкам',
    'Процент не подписанных ЭТрН по завершенным поездкам'
    ], axis=1)

final_groups_briefly

,Дивизион,Общее количество межскладских перемещений,Кол-во ЭТрН,Процент кол-ва ЭТрН к общему кол-ву перемещений,ЭТрН подписаны по завершенным поездкам,ЭТрН не подписаны по завершенным поездкам
0,?,39,5,13,0% (0 шт),0% (0 шт)
1,Восток,4,0,0,0% (0 шт),0% (0 шт)
2,Восточная Сибирь,37,13,35,0% (0 шт),48% (12 шт)
3,Север,95,91,96,93% (84 шт),7% (6 шт)
4,Центр,54,39,72,4% (2 шт),74% (35 шт)
5,Итого,229,148,65,48% (86 шт),29% (53 шт)


In [ ]:
# приведем к процентному виду ряд столбцов в некоторых таблицах
cols = [
    'Процент кол-ва ЭТрН к общему кол-ву перемещений',
    'Процент подписанных ЭТрН по завершенным поездкам',
    'Процент не подписанных ЭТрН по завершенным поездкам'
]

final_table[cols] = final_table[cols].astype(str) + '%'

final_groups_briefly['Процент кол-ва ЭТрН к общему кол-ву перемещений'] = (
    final_groups_briefly['Процент кол-ва ЭТрН к общему кол-ву перемещений'].astype(str) + '%'
)

final_groups['Процент кол-ва ЭТрН к общему кол-ву перемещений'] = (
    final_groups['Процент кол-ва ЭТрН к общему кол-ву перемещений'].astype(str) + '%'
)

# Сохранение результатов в файл Excel

In [ ]:
# данные для листов с одной таблицей
datasets_single = {
    'СВОД полный': final_groups,
    'СВОД краткий': final_groups_briefly,
    'подписано': archive_mod,
    'титул 1': title_1_mod,
    'титул 2': title_2_mod,
    'титул 3': title_3_mod,
    'титул 4': title_4_mod,
    'ЭД_М11': m11
    }

# данные для листов с несколькими таблицами
datasets_multiple = {'счет (2)': [
    divs_table,
    archive_gr,
    title_1_gr,
    title_2_gr,
    title_3_gr,
    title_4_gr,
    final_table
    ]}

# наименование отчета
excel_filename = 'Отчет ЭТрН.xlsx'

# создаем объект для записи в Excel
with pd.ExcelWriter(excel_filename) as writer:

  # обработка листов с одиночными таблицами
  for sheet_name, data in datasets_single.items():
    df = pd.DataFrame(data)
    df.to_excel(writer, sheet_name=sheet_name, header=True, index=False)

  # обработка листов с несколькими таблицами
  for sheet_name, tables_list in datasets_multiple.items():
    start_row = 0
    for table_data in tables_list:
      df = pd.DataFrame(table_data)

      # размещаем каждую таблицу последовательно сверху вниз
      df.to_excel(writer, sheet_name=sheet_name, startrow=start_row, startcol=0, header=True, index=False)

      # определяем новую позицию для следующей таблицы - длина таблицы + 2 пустых ряда вниз
      start_row += len(df) + 2